# Purpose

The purpose of this notebook is to calculate the # of residues targetable by variant base editors.


In [2]:
import pandas as pd 
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import log
from os import path
import csv

/Users/asangree/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#PAM list 4 for NG
ng_pams = pd.read_csv('../../Data_files/2020-10-23_PAM list 4.csv')


In [4]:
ng_pam_list = ng_pams['PAM'].tolist()
print(len(ng_pam_list))

62


In [5]:
#PAM list 5 for spg
spg_pams = pd.read_csv('../../Data_files/2021-04-06_PAM list 5.csv')

In [6]:
spg_pam_list = spg_pams['PAM'].tolist()
print(len(spg_pam_list))

65


## CBE 

In [7]:
brca1_cbe = pd.read_table('../../Data_files/BRCA1/annotations/brca1_ng base-editor_tiling_library.txt')

In [8]:
print(brca1_cbe.shape)
brca1_cbe.head()

(11704, 25)


,sgRNA sequence,sgRNA context sequence,Gene Symbol,Ensembl Gene ID,Ensembl transcript ID,Gene strand,Genome assembly,Transcript reference allele,Transcript alternate allele,Genome reference allele,...,Edit,# edits,#silent edits,Nucleotide edits,Amino acid edits,Mutation category,Clinical significance,BsmBI flag,4T flag,Off target count
0,CTGTTCCAATGAACTTTAAC,CTTTCTGTTCCAATGAACTTTAACACATTAG,BRCA1,ENSG00000012048,ENST00000471181,-1.0,GRCh38,G,A,C,...,C-T,2.0,0.0,C_6;C_7;,utr;utr;,UTR;UTR;,None;Uncertain significance,NaN,NaN,2.0
1,ATGTGTTAAAGTTCATTGGA,TCTAATGTGTTAAAGTTCATTGGAACAGAAA,BRCA1,ENSG00000012048,ENST00000471181,-1.0,GRCh38,C,T,G,...,C-T,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,TCTGTTCCAATGAACTTTAA,TCTTTCTGTTCCAATGAACTTTAACACATTA,BRCA1,ENSG00000012048,ENST00000471181,-1.0,GRCh38,G,A,C,...,C-T,2.0,0.0,C_7;C_8;,utr;utr;,UTR;UTR;,Uncertain significance;None,NaN,NaN,6.0
3,TGTGTTAAAGTTCATTGGAA,CTAATGTGTTAAAGTTCATTGGAACAGAAAG,BRCA1,ENSG00000012048,ENST00000471181,-1.0,GRCh38,C,T,G,...,C-T,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
4,TTCTGTTCCAATGAACTTTA,TTCTTTCTGTTCCAATGAACTTTAACACATT,BRCA1,ENSG00000012048,ENST00000471181,-1.0,GRCh38,G,A,C,...,C-T,1.0,0.0,C_8;,utr;,UTR;,None,NaN,NaN,3.0


In [9]:
brca1_cbe = brca1_cbe[brca1_cbe['Gene Symbol'] == 'BRCA1']

In [10]:
brca1_cbe['N_PAM'] = ['N'+p[1:3]+'N'  for p in brca1_cbe['PAM']]

In [11]:
def GetMostSevereMutationType(string):
    if type(string) == float:
        return 'No edits'
    elif 'Nonsense' in string:
        return 'Nonsense'
    elif 'Splice-acceptor' in string or 'Splice-donor' in string:
        return 'Splice site'
    elif 'Missense' in string:
        return 'Missense'
    elif 'Intron' in string:
        return 'Intron'        
    elif 'Silent' in string:
        return 'Silent'
    elif 'UTR' in string:
        return 'UTR'

In [12]:
def GetResidues(string):
    new_string = ''
    if type(string) != float:
        edits = string.split(';')
        for edit in edits:
            if edit.startswith('Exon'):
                new_string += 'intron;'
            elif edit == 'utr':
                new_string += 'utr;'
            elif edit == '':
                continue
            else:
                # Strip off all non-digit characters
                for character in edit:
                    if character.isdigit():        
                        new_string += character
                new_string += ';'
    return new_string

In [13]:
brca1_cbe['Mutation bin'] = brca1_cbe['Mutation category'].apply(GetMostSevereMutationType)


In [14]:
print(len(brca1_cbe[brca1_cbe['N_PAM'] == 'NGGN']))

534


In [15]:
brca1_cbe['Residues'] = brca1_cbe['Amino acid edits'].apply(GetResidues)


In [16]:
all_pams = set(brca1_cbe['PAM'].tolist())
print(len(all_pams))

253


In [17]:
#cbe
def get_residues(df):
    res = []
    count = 0
    for i,r in df.iterrows():
        edits = r['Amino acid edits'].split(';')
        for edit in edits:
            if edit != 'utr':
                if not edit.startswith('Exon'):
                    if edit[0:3] != edit[-3:]: 
                        res.append(edit[0:-3])

    res = list(set(res))
    print(len(res))
    #df = pd.DataFrame(res)
    #df.to_csv('wt_calc_test_scores.csv')
    
print ('NGG-CBE')
get_residues(brca1_cbe[(brca1_cbe['Gene Symbol'] == 'BRCA1')  & 
                   (brca1_cbe['N_PAM'] == 'NGGN') & 
                   (brca1_cbe['Mutation bin'].isin(['Intron','UTR','Missense','Nonsense','Silent','Splice site']))])

print ('NG-CBE')
get_residues(brca1_cbe[(brca1_cbe['Gene Symbol'] == 'BRCA1')  & 
                   (brca1_cbe['PAM'].isin(ng_pam_list)) & 
                   (brca1_cbe['Mutation bin'].isin(['Intron','UTR','Missense','Nonsense','Silent','Splice site']))])

print ('SpG-CBE')
get_residues(brca1_cbe[(brca1_cbe['Gene Symbol'] == 'BRCA1')  & 
                   (brca1_cbe['PAM'].isin(spg_pam_list)) & 
                   (brca1_cbe['Mutation bin'].isin(['Intron','UTR','Missense','Nonsense','Silent','Splice site']))])

print ('total residues')
get_residues(brca1_cbe[(brca1_cbe['Gene Symbol'] == 'BRCA1')  &  
                  (brca1_cbe['Mutation bin'].isin(['Intron','UTR','Missense','Nonsense','Silent','Splice site']))])

print ('any PAM with CBE')
get_residues(brca1_cbe[(brca1_cbe['Gene Symbol'] == 'BRCA1')  & 
                   (brca1_cbe['PAM'].isin(all_pams)) & 
                   (brca1_cbe['Mutation bin'].isin(['Intron','UTR','Missense','Nonsense','Silent','Splice site']))])


NGG-CBE
223
NG-CBE
777
SpG-CBE
837
total residues
1241
any PAM with CBE
1241


## ABE 

In [18]:
brca1_abe = pd.read_csv('../../Data_files/BRCA1/annotations/2021-04-28_BRCA1_ABE_annotation_v2.csv')

In [19]:
brca1_abe = brca1_abe[brca1_abe['Gene Symbol'] == 'BRCA1']

In [20]:
print(brca1_abe.shape)
brca1_abe.head()

(11524, 25)


,Unnamed: 0,sgRNA sequence,Gene Symbol,sgRNA context sequence,Ensembl Gene ID,Ensembl transcript ID,Gene strand,Genome assembly,Transcript reference allele,Transcript alternate allele,...,PAM,Edit,# edits,#silent edits,Nucleotide edits,Amino acid edits,Mutation category,Clinical significance,BsmBI flag,4T flag
0,0,CTGTTCCAATGAACTTTAAC,BRCA1,CTTTCTGTTCCAATGAACTTTAACACATTAG,ENSG00000012048,ENST00000471181,-1.0,GRCh38,T,C,...,ACAT,A-G,1.0,0.0,A_8;,utr;,UTR;,Uncertain significance,NaN,NaN
1,1,ATGTGTTAAAGTTCATTGGA,BRCA1,TCTAATGTGTTAAAGTTCATTGGAACAGAAA,ENSG00000012048,ENST00000471181,-1.0,GRCh38,A,G,...,ACAG,A-G,1.0,0.0,A_8;,Exon2:-23;,Intron;,None,NaN,NaN
2,2,TCTGTTCCAATGAACTTTAA,BRCA1,TCTTTCTGTTCCAATGAACTTTAACACATTA,ENSG00000012048,ENST00000471181,-1.0,GRCh38,T,C,...,CACA,A-G,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,TGTGTTAAAGTTCATTGGAA,BRCA1,CTAATGTGTTAAAGTTCATTGGAACAGAAAG,ENSG00000012048,ENST00000471181,-1.0,GRCh38,A,G,...,CAGA,A-G,2.0,0.0,A_7;A_8;,Exon2:-23;Exon2:-22;,Intron;Intron;,None;Conflicting interpretations of pathogenicity,NaN,NaN
4,4,TTCTGTTCCAATGAACTTTA,BRCA1,TTCTTTCTGTTCCAATGAACTTTAACACATT,ENSG00000012048,ENST00000471181,-1.0,GRCh38,T,C,...,ACAC,A-G,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
brca1_abe['N_PAM'] = ['N'+p[1:3]+'N'  for p in brca1_abe['PAM']]

In [22]:
brca1_abe['Mutation bin'] = brca1_abe['Mutation category'].apply(GetMostSevereMutationType)


In [80]:
gene = 'BRCA1'
def get_residues(df):
    res = []
    count = 0
    for i,r in df.iterrows():
        edits = r['Amino acid edits'].split(';')
        for edit in edits:
            if edit != 'utr':
                if not edit.startswith('Exon'):
                    if edit[0:3] != edit[-3:]: 
                        res.append(edit[0:-3])

    res = list(set(res))
    print(len(res))

print ('NGG-ABE')
get_residues(brca1_abe[(brca1_abe['Gene Symbol'] == gene)  & 
                   (brca1_abe['N_PAM'] == 'NGGN') & 
                   (brca1_abe['Mutation bin'].isin(['Missense','Nonsense','Splice site']))])
print ('NG-ABE')
get_residues(brca1_abe[(brca1_abe['Gene Symbol'] == gene)  & 
                   (brca1_abe['PAM'].isin(ng_pam_list)) & 
                   (brca1_abe['Mutation bin'].isin(['Missense','Nonsense','Splice site']))])

print ('SpG-ABE')
get_residues(brca1_abe[(brca1_abe['Gene Symbol'] == gene)  & 
                   (brca1_abe['PAM'].isin(spg_pam_list)) & 
                   (brca1_abe['Mutation bin'].isin(['Missense','Nonsense','Splice site']))])

print ('total residues')
get_residues(brca1_abe[(brca1_abe['Gene Symbol'] == 'BRCA1')  &  
                  (brca1_abe['Mutation bin'].isin(['Missense','Nonsense','Splice site']))])

print ('any PAM with ABE')
get_residues(brca1_abe[(brca1_abe['Gene Symbol'] == gene)  & 
                   (brca1_abe['PAM'].isin(all_pams)) & 
                   (brca1_abe['Mutation bin'].isin(['Missense','Nonsense','Splice site']))])




NGG-ABE
308
NG-ABE
992
SpG-ABE
1060
total residues
1502
any PAM with ABE
1502


## Calculate the residues that are targetable by either an ABE or CBE for each of the Cas enzymes

to calculate the # of residues editable with either abe or cbe, add up the # each can target, then subtract the length of the intersection 

In [ ]:
#change the df such that it outputs the residues as a df

In [45]:
col_name = ['Residue']
def get_residues_v2(df):
    res = []
    count = 0
    for i,r in df.iterrows():
        edits = r['Amino acid edits'].split(';')
        for edit in edits:
            if edit != 'utr':
                if not edit.startswith('Exon'):
                    if edit[0:3] != edit[-3:]: 
                        res.append(edit[0:-3])

    res = list(set(res))
    print(len(res))
    df = pd.DataFrame(res, columns= col_name)
    return df

In [77]:
ngg_cbe_df = get_residues_v2(brca1_cbe[(brca1_cbe['Gene Symbol'] == 'BRCA1')  & 
                   (brca1_cbe['N_PAM'] == 'NGGN') & 
                   (brca1_cbe['Mutation bin'].isin(['Missense','Nonsense','Splice site']))])


192


In [47]:
ngg_cbe_df

,Residue
0,Asp1872
1,Asp1496
2,Pro1797
3,Gly817
4,Asp825
...,...
218,Pro1833
219,Asp201
220,Asp67
221,Thr1794


In [82]:
ng_cbe_df = get_residues_v2(brca1_cbe[(brca1_cbe['Gene Symbol'] == 'BRCA1')  & 
                   (brca1_cbe['PAM'].isin(ng_pam_list)) & 
                   (brca1_cbe['Mutation bin'].isin(['Missense','Nonsense','Silent','Splice site']))])



777


In [83]:
spg_cbe_df = get_residues_v2(brca1_cbe[(brca1_cbe['Gene Symbol'] == 'BRCA1')  & 
                   (brca1_cbe['PAM'].isin(spg_pam_list)) & 
                   (brca1_cbe['Mutation bin'].isin(['Missense','Nonsense','Silent','Splice site']))])


837


In [87]:
pamless_cbe_df = get_residues_v2(brca1_cbe[(brca1_cbe['Gene Symbol'] == 'BRCA1')  &  
                  (brca1_cbe['Mutation bin'].isin(['Missense','Nonsense','Silent','Splice site']))])

1241


In [84]:
ngg_abe_df = get_residues_v2(brca1_abe[(brca1_abe['Gene Symbol'] == 'BRCA1')  & 
                   (brca1_abe['N_PAM'] == 'NGGN') & 
                   (brca1_abe['Mutation bin'].isin(['Missense','Nonsense','Silent','Splice site']))])


308


In [85]:
ng_abe_df = get_residues_v2(brca1_abe[(brca1_abe['Gene Symbol'] == 'BRCA1')  & 
                   (brca1_abe['PAM'].isin(ng_pam_list)) & 
                   (brca1_abe['Mutation bin'].isin(['Missense','Nonsense','Silent','Splice site']))])



992


In [86]:
spg_abe_df = get_residues_v2(brca1_abe[(brca1_abe['Gene Symbol'] == 'BRCA1')  & 
                   (brca1_abe['PAM'].isin(spg_pam_list)) & 
                   (brca1_abe['Mutation bin'].isin(['Missense','Nonsense','Silent','Splice site']))])


1060


In [88]:
pamless_abe_df = get_residues_v2(brca1_abe[(brca1_abe['Gene Symbol'] == 'BRCA1')  &  
                  (brca1_abe['Mutation bin'].isin(['Missense','Nonsense','Silent','Splice site']))])

1502


NGG - WT accessible residues

In [54]:
ngg_cbe_res = list(set(ngg_cbe_df['Residue']))
ngg_abe_res = list(set(ngg_abe_df['Residue']))

In [64]:
ngg_res = list((ngg_cbe_res+ngg_abe_res))
print(len(ngg_res))

531


In [57]:
ngg_res_unique = list(set(ngg_cbe_res+ngg_abe_res))
print(len(ngg_res_unique))

460


In [72]:
460/1885

0.2440318302387268

460 unique residues with WT-ABE or CBE, or 24.4% of protein

NG- accessible residues

In [66]:
ng_cbe_res = list(set(ng_cbe_df['Residue']))
ng_abe_res = list(set(ng_abe_df['Residue']))

In [68]:
ng_res = list((ng_cbe_res+ng_abe_res))
print(len(ng_res))

1769


In [67]:
ng_res_unique = list(set(ng_cbe_res+ng_abe_res))
print(len(ng_res_unique))

1347


In [73]:
1347/1885

0.7145888594164457

1347 unique residues with NG-ABE or CBE, or 71.5% of protein

SpG accessible residues

In [69]:
spg_cbe_res = list(set(spg_cbe_df['Residue']))
spg_abe_res = list(set(spg_abe_df['Residue']))

In [70]:
spg_res = list((spg_cbe_res+spg_abe_res))
print(len(spg_res))

1897


In [71]:
spg_res_unique = list(set(spg_cbe_res+spg_abe_res))
print(len(spg_res_unique))

1423


In [74]:
1423/1885

0.7549071618037135

1423 unique residues with SpG-ABE or CBE, or 75.5% of protein

PAMless

In [89]:
pamless_cbe_res = list(set(pamless_cbe_df['Residue']))
pamless_abe_res = list(set(pamless_abe_df['Residue']))

In [90]:
pamless_res_unique = list(set(pamless_cbe_res+pamless_abe_res))
print(len(pamless_res_unique))

1810


In [91]:
1810/1885

0.9602122015915119